## Building a Recommendation Classifier

<h4>Let's start by loading the data as in the first lab.

In [63]:
import os 
os.getcwd()
os.chdir(r'C:\Users\MAIN\Desktop\ML\Internship SMinds\Recommender')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [64]:
#import dataset
df = pd.read_csv("TripAdvReview.csv", na_values=" ")

In [65]:
#remove extra columns and keep the necessary ones for analysis
df = pd.DataFrame(df.drop(['Bubble_Count', 'Review_Count'], axis=1))
df.columns
print(df.head(n=5))

                                               Title  Rating  Review_counts  \
0                            Vacation/Cottage Rental     5.0           85.0   
1    Mistiso's Place Vacation Rentals- Purcell Suite     4.5          100.0   
2  Toronto 4 Bedroom House Spacious Clean Beautif...     5.0           36.0   
3                                   Our Sweet Escape     5.0           34.0   
4    Walk to ocean from Lyons Cottage Rentals in PEI     4.5           48.0   

             City               Province  \
0   Niagara Falls                Ontario   
1          Nelson       British Columbia   
2         Toronto                Ontario   
3  Qualicum Beach       British Columbia   
4        Stanhope   Prince Edward Island   

                                             Reviews  
0  1 mention of vacation rentals\nThis rental fel...  
1  4 mentions of vacation rentals\nI can't say en...  
2  1 mention of vacation rentals\nback yard! Woul...  
3  3 mentions of vacation rentals\nenjoy

In [66]:
#remove new line characters from variable columns
df = df.replace(r'\n',' ', regex=True)

#remove the numbers in the review column
df.Reviews = df.Reviews.str.replace('\d+', '')

In [67]:
#fills the rating and Review_counts variable missing values with the mean and median respectively
df = df.fillna({'Rating': df.Rating.median(), 'Review_counts': df.Review_counts.mean()})

In [68]:
#drop all missing values
df.dropna(axis=0,how='any', inplace= True)
df.head(n=100)

,Title,Rating,Review_counts,City,Province,Reviews
0,Vacation/Cottage Rental,5.0,85.0,Niagara Falls,Ontario,mention of vacation rentals This rental felt ...
1,Mistiso's Place Vacation Rentals- Purcell Suite,4.5,100.0,Nelson,British Columbia,mentions of vacation rentals I can't say enou...
2,Toronto 4 Bedroom House Spacious Clean Beautif...,5.0,36.0,Toronto,Ontario,mention of vacation rentals back yard! Would ...
3,Our Sweet Escape,5.0,34.0,Qualicum Beach,British Columbia,mentions of vacation rentals enjoyable. I hav...
4,Walk to ocean from Lyons Cottage Rentals in PEI,4.5,48.0,Stanhope,Prince Edward Island,mentions of vacation rentals quiet morning as...
5,Best views in Nelson,5.0,33.0,Nelson,British Columbia,mentions of vacation rentals to buy even the ...
6,"Shale Beach House, Blue Mountain Collingwood Ont.",5.0,77.0,Blue Mountains,Ontario,mention of vacation rentals vacation spot! Th...
7,Paradise Vacation Rentals,5.0,25.0,North Bay,Ontario,This has to be one of the best vacation places...
8,Downtown Toronto 4BR Townhouse,5.0,55.0,Toronto,Ontario,mentions of vacation rentals paint but you co...
9,Les Rives du Sanctuaire,5.0,53.0,Mont Tremblant,Quebec,mentions of vacation rentals spend majestic v...


In [69]:
df.index=range(571)

In [70]:
print(df.shape)
df.isnull().sum()

(571, 6)


Title            0
Rating           0
Review_counts    0
City             0
Province         0
Reviews          0
dtype: int64

In [71]:
# Let's understand the two lists: reviews (text_train) and their labels (y_train)
print("Type : ",type(df.Reviews))
print("Length of reviews: ",len(df.Reviews))
print("Review at index 6:\n ", df.Reviews[6])
print("Label of the review at Index 6: ",df.Rating[6])
# The ratings laels is a continous series of float numbers

Type :  <class 'pandas.core.series.Series'>
Length of reviews:  571
Review at index 6:
   mention of vacation rentals vacation spot! There are three levels and so each family got their very own
Label of the review at Index 6:  5.0


In [72]:
#import libraries
import nltk
import re
from nltk.corpus import stopwords

In [73]:
#filter punctuations, stemming and stopwords
corpus = []
for i in range(len(df)):
    review = re.sub('[^a-zA-Z0-9]', ' ', df['Reviews'][i])
    review = review.lower()
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    stemmer = nltk.PorterStemmer()
    lemmatizer = nltk.WordNetLemmatizer()
    tokens_list = tokenizer.tokenize(review)
    tokens = []
    for token in tokens_list:
        tokens.append(lemmatizer.lemmatize(token))
        stop_words = stopwords.words("english")
    filtered_words = [w for w in tokens if w not in stop_words]
    review = ' '.join(filtered_words)
    corpus.append(review)

In [74]:
#Bag of Words model to convert corpus into X
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(corpus)
key = list(cv.vocabulary_.keys())
key.sort()
X = pd.DataFrame(cv.transform(corpus).toarray(),columns = key)
y = df.Rating

#TF_IDF model to convert corpus into X
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

X2 = pd.DataFrame(tfidf.fit_transform(corpus).toarray())

In [75]:
#Label Encoding for Categorical Target Variable

from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
df['Rating'] = lb.fit_transform(df['Rating'])
y=df.Rating
Rating = df.Rating

In [76]:
df.Rating.value_counts()

6    372
5    149
4     31
3      8
2      5
0      4
1      2
Name: Rating, dtype: int64

 <h4>We need to get unique words to determine the vocabulary size

In [77]:
reviews = df.Reviews

In [96]:
uniq_words=set()
for doc in reviews:
    for word in doc.split(" "):
        uniq_words.add(word)
vocab_size=len(uniq_words)
print ("Total Unique words:",vocab_size)



Total Unique words: 1856


<h4>Time to import Keras and tensorflow modules

In [97]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding



<h4>We need to convert each of the words in the reviews to one-hot vectors. Below is the code to get integer indexes of the words for one hot vector. Note that we don't need to store all zeros as only the integer index for the word in a vector will have a value of 1.

In [98]:
# Integer encode the documents

encoded_reviews = [one_hot(review, vocab_size) for review in reviews]
print(encoded_reviews[1])

[1839, 154, 1210, 305, 861, 1354, 1688, 710, 1302, 1502, 1201, 384, 764, 1210, 462]


<h4>We also need to know in advance the length of the longest review. This information is needed to fix the size of the embedding matrix. We leave this as an exercise for you to determine the largest length of the review. We are curently fixing the maximum length to 100 words.

In [99]:
# pad documents to a max length of n words
max_length = 50
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')
print(padded_reviews)


[[1468  154 1210 ...    0    0    0]
 [1839  154 1210 ...    0    0    0]
 [1468  154 1210 ...    0    0    0]
 ...
 [1468  154 1210 ...    0    0    0]
 [1468  154 1210 ...    0    0    0]
 [ 307 1015 1301 ...    0    0    0]]


<H4>We have completed our pre-processing, it is now time to build the neural network based classifier. But before we do that, let's split the reviews into training and test set.

In [100]:
#train_rev=padded_reviews[0:1000]
#train_lbls=Rating[0:1000]
#test_rev=padded_reviews[1001:]
#test_lbls=Rating[1001:]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_reviews,Rating,test_size=0.3, random_state=0)


<h4>Now we need to define the basics of model for neural network. 

In [101]:
from keras import models
# define the model
model = Sequential()
# Define the embedding matrix dimensions. Each vector is of 8 dimensions and there will be total of vocab_size vectors
# The input length (window) is 100 words so the output from embedding layer will be a conactenated (flattened) vector of 
# 800 dimensions
model.add(Embedding(vocab_size, 16, input_length=max_length))
model.add(Flatten())

model.add(Dense(units=3, activation='sigmoid'))
model.add(Dense(units=2, activation='sigmoid'))
model.add(Dense(units=1, activation='sigmoid'))
# compile the model with stochastic gradient descent and binary cross entropy
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 50, 16)            29696     
_________________________________________________________________
flatten_8 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 3)                 2403      
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 8         
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 3         
Total params: 32,110
Trainable params: 32,110
Non-trainable params: 0
_________________________________________________________________
None


In [102]:
# Fit the model... there are few docs, so I am trying with batch_size=1, you can delete it for default batch 
#size or change it to a bigger number
model.fit(X_train, y_train, epochs=200,batch_size=30, verbose=1)

Epoch 1/200
399/399 [==============================] - 0s 1ms/step - loss: -1.4507 - acc: 0.0000e+00
Epoch 2/200
399/399 [==============================] - 0s 63us/step - loss: -6.6763 - acc: 0.0025
Epoch 3/200
399/399 [==============================] - 0s 70us/step - loss: -12.2260 - acc: 0.0025
Epoch 4/200
399/399 [==============================] - 0s 70us/step - loss: -18.0758 - acc: 0.0025
Epoch 5/200
399/399 [==============================] - 0s 68us/step - loss: -23.7948 - acc: 0.0025
Epoch 6/200
399/399 [==============================] - 0s 63us/step - loss: -29.4822 - acc: 0.0025
Epoch 7/200
399/399 [==============================] - 0s 73us/step - loss: -35.1120 - acc: 0.0025
Epoch 8/200
399/399 [==============================] - 0s 90us/step - loss: -40.7576 - acc: 0.0025
Epoch 9/200
399/399 [==============================] - 0s 98us/step - loss: -46.4024 - acc: 0.0025
Epoch 10/200
399/399 [==============================] - 0s 73us/step - loss: -52.0693 - acc: 0.0025
Epoch 11

399/399 [==============================] - 0s 73us/step - loss: -71.3212 - acc: 0.0025
Epoch 84/200
399/399 [==============================] - 0s 70us/step - loss: -71.3212 - acc: 0.0025
Epoch 85/200
399/399 [==============================] - 0s 60us/step - loss: -71.3212 - acc: 0.0025
Epoch 86/200
399/399 [==============================] - 0s 60us/step - loss: -71.3212 - acc: 0.0025
Epoch 87/200
399/399 [==============================] - 0s 55us/step - loss: -71.3212 - acc: 0.0025
Epoch 88/200
399/399 [==============================] - 0s 53us/step - loss: -71.3212 - acc: 0.0025
Epoch 89/200
399/399 [==============================] - 0s 63us/step - loss: -71.3212 - acc: 0.0025
Epoch 90/200
399/399 [==============================] - 0s 55us/step - loss: -71.3212 - acc: 0.0025
Epoch 91/200
399/399 [==============================] - 0s 53us/step - loss: -71.3212 - acc: 0.0025
Epoch 92/200
399/399 [==============================] - 0s 63us/step - loss: -71.3212 - acc: 0.0025
Epoch 93/200


399/399 [==============================] - 0s 60us/step - loss: -71.3212 - acc: 0.0025
Epoch 165/200
399/399 [==============================] - 0s 58us/step - loss: -71.3212 - acc: 0.0025
Epoch 166/200
399/399 [==============================] - 0s 55us/step - loss: -71.3212 - acc: 0.0025
Epoch 167/200
399/399 [==============================] - 0s 50us/step - loss: -71.3212 - acc: 0.0025
Epoch 168/200
399/399 [==============================] - 0s 53us/step - loss: -71.3212 - acc: 0.0025
Epoch 169/200
399/399 [==============================] - 0s 55us/step - loss: -71.3212 - acc: 0.0025
Epoch 170/200
399/399 [==============================] - 0s 55us/step - loss: -71.3212 - acc: 0.0025
Epoch 171/200
399/399 [==============================] - 0s 58us/step - loss: -71.3212 - acc: 0.0025
Epoch 172/200
399/399 [==============================] - 0s 53us/step - loss: -71.3212 - acc: 0.0025
Epoch 173/200
399/399 [==============================] - 0s 55us/step - loss: -71.3212 - acc: 0.0025
Epoc

<h4>Now, we shall evaluate our model against the test set that we kep separate earlier.

In [103]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print('Accuracy: %f' % (accuracy*100))

172/172 [==============================] - 0s 813us/step
Accuracy: 0.581395


<h4>Bonus: Precision and Recall

In [104]:
from sklearn.metrics import classification_report
import numpy as np

predictions = model.predict(X_test, batch_size=100, verbose=1)
predictions_bool = np.argmax(predictions, axis=1)

print(classification_report(y_test, predictions_bool))

172/172 [==============================] - 0s 720us/step
              precision    recall  f1-score   support

           0       0.01      1.00      0.01         1
           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         9
           5       0.00      0.00      0.00        41
           6       0.00      0.00      0.00       117

   micro avg       0.01      0.01      0.01       172
   macro avg       0.00      0.17      0.00       172
weighted avg       0.00      0.01      0.00       172



C:\Users\MAIN\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\MAIN\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\MAIN\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
